AutoML framework prediction



In [1]:
# # !pip install matplotlib
# Nie da się używać MLJAR i Autosklearn jednocześnie
# !pip install mljar-supervised
# !pip uninstall -y Cython scipy pyparsing scikit_learn imbalanced-learn mlxtend yellowbrick

# # 2. install packages to be downgraded
# !pip install Cython==0.29.36 scipy==1.9 pyparsing==2.4

# # 3. install older scikit-learn disregarding its dependencies
# !pip install scikit-learn==0.24.2 --no-build-isolation

# # 4. finally install auto-sklearn
# !pip install auto-sklearn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score
import os
import warnings
warnings.filterwarnings('ignore')
# from autosklearn.classification import AutoSklearnClassifier
# from autosklearn.experimental.askl2 import AutoSklearn2Classifier
# from autosklearn.metrics import roc_auc
from supervised.automl import AutoML

Read data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir = "/content/drive/MyDrive/Colab Notebooks/"

In [ ]:
X = pd.read_csv(dir + 'data/artificial_train.data', sep=' ', header = None).drop(500, axis=1)
#chosen
# leader = pd.read_csv(dir + 'model_best20min/16_CatBoost/learner_fold_0_importance.csv')
# chosen = leader.loc[leader['mean_importance'] != 0,'feature'].tolist()
# X = X.loc[:,chosen]
y = pd.read_csv(dir + 'data/artificial_train.labels', header = None)
y.replace(-1, 0, inplace=True)
y = np.array(y).ravel()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

MLJAR training

In [ ]:
# MLJAR 1
automl1 = AutoML(results_path=dir + 'model_10min/',
                algorithms=['Xgboost', 'CatBoost', 'Extra Trees'],
                mode='Compete',
                eval_metric='accuracy',
                total_time_limit=600,
                random_state=42,
                explain_level=1,
                features_selection=True,
                golden_features=False)
# automl1.fit(X_train, y_train)
# MLJAR 2
automl2 = AutoML(results_path=dir + 'model_20min',
                algorithms=['Xgboost', 'CatBoost', 'Extra Trees'],
                mode='Compete',
                eval_metric='accuracy',
                total_time_limit=1200,
                random_state=42,
                explain_level=1,
                features_selection=True,
                golden_features=False)
# automl2.fit(X_train, y_train)
# MLJAR 3

automl3 = AutoML(results_path=dir + 'modelchosen/',
                algorithms=['Xgboost', 'CatBoost', 'Extra Trees'],
                mode='Compete',
                eval_metric='accuracy',
                total_time_limit=734,
                random_state=42,
                explain_level=1,
                features_selection=True,
                golden_features=False)
#chosen
# leader = pd.read_csv(dir + 'model_20min/16_CatBoost/learner_fold_0_importance.csv')
# chosen = leader.loc[leader['mean_importance'] != 0,'feature'].tolist()
# X = X.loc[:,chosen]
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# automl3.fit(X_train, y_train)

Autosklearn training

In [ ]:
from autosklearn.metrics import balanced_accuracy
settings = {
  "time_left_for_this_task": 600,  # seconds
  "seed": 42,
  "metric": balanced_accuracy,
  "n_jobs": 4
}


resampling_strategy = "holdout"

In [ ]:


##autosklearn, model 1
auto_pre_no = AutoSklearnClassifier(
    **settings,
    resampling_strategy=resampling_strategy,
)
#autosklearn, model 2
auto_pre_tree = AutoSklearnClassifier(
    **settings,
    resampling_strategy=resampling_strategy,
    include = {
        'classifier': ["random_forest", "extra_trees"],
    },
    exclude=None
)
auto_pre_no.fit(X_train, y_train)

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      metric=balanced_accuracy, n_jobs=4,
                      per_run_time_limit=240, seed=42,
                      time_left_for_this_task=600)

In [ ]:
preprocessors2 = []
for i, (weight, pipeline) in enumerate(auto_pre_no.get_models_with_weights()):
    for stage_name, component in pipeline.named_steps.items():
        if "feature_preprocessor" in stage_name:
          print(i)
          print(component.choice)
          #preprocessors2.append(component.choice.preprocessor)

0
autosklearn.pipeline Nystroem kernel approximation
1
autosklearn.pipeline Extra Trees Classifier Preprocessing
2
autosklearn.pipeline Random Kitchen Sinks
3
autosklearn.pipeline Fast Independent Component Analysis
4
autosklearn.pipeline Extra Trees Classifier Preprocessing
5
autosklearn.pipeline Nystroem kernel approximation
6
autosklearn.pipeline Random Trees Embedding


In [ ]:
leaderboard = auto_pre_no.leaderboard(sort_by="model_id", ensemble_only=True)
print(leaderboard)
auto_pre_no.sprint_statistics()

          rank  ensemble_weight           type      cost   duration
model_id                                                           
9            1             0.06    extra_trees  0.126894  26.734182
24           5             0.02            lda  0.429924  14.872607
26           7             0.02  random_forest  0.494318  17.678612
27           2             0.30     libsvm_svc  0.399621   9.524431
34           4             0.12            qda  0.409091   9.990599
52           6             0.34            qda  0.435606  14.576446
54           3             0.14            qda  0.407197  11.077237


'auto-sklearn results:\n  Dataset name: 732020f5-b217-11ee-95d2-0242ac1c000c\n  Metric: balanced_accuracy\n  Best validation score: 0.873106\n  Number of target algorithm runs: 59\n  Number of successful target algorithm runs: 50\n  Number of crashed target algorithm runs: 0\n  Number of target algorithms that exceeded the time limit: 4\n  Number of target algorithms that exceeded the memory limit: 5\n'

Save metrics and results

In [ ]:
auto_10 = AutoML(results_path=dir + 'model_best/')

In [ ]:
# res_df = pd.DataFrame(columns=['train_roc', 'train_bacc', 'valid_roc', 'valid_bacc'])

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
y_probabilities = auto_10.predict_proba(X_valid)[:, 1]  # Use the probabilities of the positive class
y_pred = auto_10.predict(X_valid)
# Compute ROC AUC score
train_roc_AUC = roc_auc_score(y_train, auto_10.predict_proba(X_train)[:, 1])
roc_AUC = roc_auc_score(y_valid, y_probabilities)
acc = accuracy_score(y_valid, y_pred)
train_bacc = balanced_accuracy_score(y_train, auto_10.predict(X_train))
bacc = balanced_accuracy_score(y_valid, auto_10.predict(X_valid))
print(f'ROC AUC Score: {roc_AUC}')
print(f'Accuracy Score: {acc}')
print('Balanced accuracy: ' + str(bacc))

ROC AUC Score: 0.9135499999999999
Accuracy Score: 0.845
Balanced accuracy: 0.845


In [ ]:
res_df.loc['auto_mljar_10min',:] = [train_roc_AUC, train_bacc, roc_AUC, bacc]

In [ ]:
res_df

,train_roc,train_bacc,valid_roc,valid_bacc
auto_mljar20min,0.992889,0.954375,0.906675,0.83
auto_mljar_87cols,0.995953,0.981875,0.930375,0.86
auto_mljar_10min,0.990355,0.94625,0.91355,0.845


In [ ]:
# autosklearn.pipeline Fast Independent Component Analysis
# 1
# autosklearn.pipeline Select Percentile Classification
# 2
# autosklearn.pipeline Extra Trees Classifier Preprocessing
# 3
# autosklearn.pipeline Random Trees Embedding
#
# model_id     rank  ensemble_weight           type      cost   duration
# 9            4             0.02  random_forest  0.494318  16.361615
# 29           2             0.36    extra_trees  0.405303   6.694720
# 40           3             0.54    extra_trees  0.429924  98.909877
# 44           1             0.08    extra_trees  0.123106  24.006901
# auto-sklearn results:\n  Dataset name: d2d67bbc-b212-11ee-95d2-0242ac1c000c\n
# Metric: balanced_accuracy\n  B
# est validation score: 0.876894\n
# Number of target algorithm runs: 48\n
#  Number of successful target algorithm runs: 39\n
#  Number of crashed target algorithm runs: 0\n
#  Number of target algorithms that exceeded the time limit: 6\n
#   Number of target algorithms that exceeded the memory limit: 3\n

Prediction on test data

In [ ]:
X_test = pd.read_csv(dir + 'data/artificial_test.data', sep=' ', header = None).drop(500, axis=1)
# X_test = X_test.loc[:, chosen] # dla jednego z modeli
pred_test = pd.DataFrame(auto_10.predict_proba(X_test), columns=['a', 'b'])

pred_test.loc[:, 'b'].to_csv(dir + 'data/pred_autosklearn_tree.txt', index=False)
pred_test.loc[:,'b']

0      0.491429
1      0.471456
2      0.494180
3      0.514964
4      0.483601
         ...   
595    0.511685
596    0.491259
597    0.522156
598    0.517316
599    0.468828
Name: b, Length: 600, dtype: float64